# Importing necessary Libraries

In [1]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Loading the CSV File

In [2]:
data=pd.read_csv("spam.csv", encoding="latin-1")
print("The columns are: ", data.columns)

The columns are:  Index(['Category', 'Message'], dtype='object')


In [3]:
data['Category']=data['Category'].map({'ham':0, 'spam':1})
print("The data after converting Spam and Ham to numbers: \n") 
data.head()

The data after converting Spam and Ham to numbers: 



,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


# Text Pre-processing

In [4]:
import re
import nltk 

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer  #lemmatizing
 
lemmatizer = WordNetLemmatizer()
corpus = []

for i in range(0, len(data)):
    review = re.sub('[^a-zA-z]', ' ', data['Message'][i])
    review = review.lower()
    review = review.split()
    
    

    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)


# Converting the Data into Bag of Words

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus).toarray()

y = pd.get_dummies((data['Category']))
y = y.iloc[:,1].values

# Train Test split and Training model with Naive Bayes Classifier

In [6]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, random_state = 0)


from sklearn.naive_bayes import MultinomialNB
Spam_clasfcn_model = MultinomialNB(alpha=.01)
Spam_clasfcn_model.fit(X_train, y_train)
y_pred = Spam_clasfcn_model.predict(X_test)

# Evaluation of the Model

In [7]:
from sklearn.metrics import confusion_matrix
confusion_m = confusion_matrix(y_test, y_pred)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy = accuracy*100
print(accuracy)

97.48878923766816


# Saving the data in '.pkl' format to be used in GUI

In [8]:
import pickle

pickle.dump(Spam_clasfcn_model, open('spam.pkl', 'wb'))
pickle.dump(cv, open('vectorizer.pkl', 'wb'))
clasf = pickle.load(open('spam.pkl', 'rb'))
